In [1]:
import import_ipynb
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import random_unitary

from qiskit.transpiler import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeWashingtonV2
from qiskit_ibm_runtime import SamplerV2

from itertools import product
from collections import defaultdict
from params import *

In [2]:
# run a test circuit and return dictionary of results
def do_run(circuit, num_shots, noise=False, seed=1):
    if noise:
        backend = FakeWashingtonV2()
    else:
        backend = AerSimulator()
    pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend, seed_transpiler = 1)
    transpiled_circuit = pass_manager.run(circuit)
    options = {"simulator": {"seed_simulator": seed}}
    sampler = SamplerV2(backend, options=options)
    job = sampler.run([transpiled_circuit], shots=num_shots)
    result = job.result()[0].data.meas.get_counts()
    return {key: result[key] for key in sorted(result)}

# Test Circuits

In [3]:
def test_circuit():
    rand_init = random_unitary(8, seed=3) 
    rand_1 = random_unitary(2, seed=1) 
    rand_2 = random_unitary(2, seed=2) 
    rand_3 = random_unitary(2, seed=3) 
    rand_4 = random_unitary(2, seed=4) 
    rand_5 = random_unitary(2, seed=5) 
    rand_6 = random_unitary(2, seed=6)
    
    qc = QuantumCircuit(3, 0)
    
    qc.unitary(rand_init, range(3), label='rand_init')
    qc.cx(0,1)
    qc.cx(2, 0) 
    qc.unitary(rand_1, 1, label='rand_1')
    qc.unitary(rand_2, 2, label='rand_2')
    qc.cx(1,2)
    qc.cx(0, 2)
    qc.unitary(rand_3, 0, label='rand_3')
    qc.unitary(rand_4, 2, label='rand_4')
    qc.cx(1,0)
    qc.cx(2, 0)
    qc.cx(2,1)
    qc.unitary(rand_5, 0, label='rand_5')
    qc.unitary(rand_6, 1, label='rand_6')
    
    U_test = qc.to_gate()
    U_test.name = "U$_{test}$"
    
    return U_test

In [4]:
def test_circuit_1():
    rand_init = random_unitary(4, seed=3) 
    rand_1 = random_unitary(2, seed=1) 
    rand_2 = random_unitary(2, seed=2) 
    
    qc = QuantumCircuit(2, 0)
    
    qc.unitary(rand_init, range(2), label='rand_init')
    qc.cx(0,1)
    qc.unitary(rand_1, 0, label='rand_1')
    qc.unitary(rand_2, 1, label='rand_2')
    
    U_test = qc.to_gate()
    U_test.name = "U$_{test 1}$"
    
    return U_test

In [5]:
def test_circuit_2():
    rand_init = random_unitary(4, seed=7) 
    rand_1 = random_unitary(2, seed=1) 
    rand_2 = random_unitary(2, seed=2) 
    rand_3 = random_unitary(2, seed=3) 
    rand_4 = random_unitary(2, seed=4) 
    rand_5 = random_unitary(2, seed=5) 
    rand_6 = random_unitary(2, seed=6)
    
    qc = QuantumCircuit(2, 0)
    
    qc.unitary(rand_init, range(2), label='rand_init')
    qc.cx(0,1)
    qc.unitary(rand_1, 0, label='rand_1')
    qc.unitary(rand_2, 1, label='rand_2')
    qc.cx(1,0)
    qc.unitary(rand_3, 0, label='rand_3')
    qc.unitary(rand_4, 1, label='rand_4')
    qc.cx(0,1)
    qc.unitary(rand_5, 0, label='rand_5')
    qc.unitary(rand_6, 1, label='rand_6')
    
    U_test = qc.to_gate()
    U_test.name = "U$_{test}$"
    
    return U_test

# Evolution Circuits

state preparation and operators

In [6]:
def superposition_state_prep(N):
    circuit = QuantumCircuit(2*N)
    mid = N-2
    for n in range(0, mid, 1):
        if n % 2 == 1:
            circuit.h(n)
        if n > 0 and (n-2) % 4 == 0:
            circuit.x(n)
    circuit.h(mid)
    circuit.x([mid+1, mid+2])
    circuit.cx(mid, mid+1)
    circuit.cx(mid+1, mid+2)
    circuit.h(mid+1)
    for n in range(mid+3, 2*N, 1):
        if n % 2 == 1:
            circuit.h(n)
        if (n-2) % 4 == 0:
            circuit.x(n)
    U_sp = circuit.to_gate()
    U_sp.name = "U$_{sp}$"
    return U_sp

In [7]:
def meson_operator():
    circuit = QuantumCircuit(3)
    circuit.x(0)
    circuit.z(1)
    circuit.x(2)
    U_meson = circuit.to_gate()
    U_meson.name = "U$_{meson}$"
    return U_meson

In [8]:
def trotter1(N, T, epsilon, mass):
    circuit = QuantumCircuit(2*N)
    for t in range(T):

        for n in range(0, 2*N, 2):
            circuit.append(fermion_mass(epsilon, mass, (-1)**(n/2+1)), [n])
    
        for l in range(1, 2*N, 2):
            circuit.append(gauge_kinetic(epsilon), [l])
  
        for n in range(0, 2*N, 4):
            circuit.append(fermion_hopping(epsilon, 1), [n, n+1, (n+2) % (2*N)])
   
        for n in range(2, 2*N, 4):
            circuit.append(fermion_hopping(epsilon, 1), [n, n+1, (n+2) % (2*N)])
     
    U_trotter = circuit.to_gate()
    U_trotter.name = "U$_{T="+str(T)+"}$"
    return U_trotter

Construct terms for Trotter evolution <br>
Gauge kinetic, fermion mass and fermion hopping terms given in eq. (2) of https://arxiv.org/abs/2305.02361 <br>
implement $U(\epsilon) = \exp[-i\epsilon H]$ where $U(t) = U(\epsilon)^{N} = \exp[-i N \epsilon H]$ where $t = \epsilon N$ <br>
`circuit.rj(theta)` implements $\exp[-i J \frac{\theta}{2}] = \exp[-i \sigma_{j} \theta]$;

In [9]:
def gauge_kinetic(epsilon):
    circuit = QuantumCircuit(1)
    circuit.rx(epsilon/2, 0)
    U_kg = circuit.to_gate()
    U_kg.name = "U$_{gk}$"
    return U_kg

In [10]:
def fermion_mass(epsilon, mass, eta):
    circuit = QuantumCircuit(1)
    circuit.rz(-epsilon*mass * eta/2, 0)
    U_m = circuit.to_gate()
    U_m.name = "U$_m$"
    return U_m

In [11]:
def fermion_hopping(epsilon, eta):
    circuit = QuantumCircuit(3)
    circuit.sxdg(1)
    circuit.s(1)
    circuit.cx(0, 1)
    circuit.sx(0)
    circuit.cx(0, 2)
    circuit.rx(-epsilon/4 * eta, 0)
    circuit.ry(-epsilon/4 * eta, 2)
    circuit.cx(0, 2)
    circuit.sxdg(0)
    circuit.cx(0, 1)
    circuit.sdg(1)
    circuit.sx(1)
    U_fh = circuit.to_gate()
    U_fh.name = "U$_{fh}$"
    return U_fh

In [12]:
def trotter_stepper(Ntsteps, Nqbits, epsilon, mass, mid):
    for i in range(Ntsteps+1):
        qc = QuantumCircuit(2*Nqbits, 0)
        qc.append(superposition_state_prep(Nqbits), range(2*Nqbits))
        qc.append(trotter1(Nqbits, i, epsilon, mass), range(2*Nqbits))
        qc.append(meson_operator(), [mid, mid+1, mid+2])
    return qc

In [13]:
trot_step_1 = trotter_stepper(1, Nqbits, epsilon, mass, mid).decompose().decompose()
trot_step_2 = trotter_stepper(2, Nqbits, epsilon, mass, mid).decompose().decompose()
trot_step_1.measure_all()
trot_step_2.measure_all()

# Statistics

In [14]:
def fermion_number(counts, mid):
    mean = 0
    total_counts = sum(counts.values())
    for s in counts:
        p = s[mid+1]
        if p == '1':
            mean += 1./total_counts * counts[s]
    return mean

In [15]:
def bootstrap_error(counts, mid, shots, seed=1):
    np.random.seed(seed)
    nshots=shots
    B = 100
    k = list(counts.keys())
    prob = [np.abs(counts[a]) for a in k]
    means = []
    for b in range(B):
        m = 0
        samples = np.random.choice(k, size=nshots, p=(prob / sum(prob)))
        for s in samples:
            p = s[mid+1]
            if p == '1' and counts[s] > 0:
                m += 1./nshots
            elif p=='1':
                m-=1./nshots
        means.append(m)
    return float(np.std(means))

# Circuit Knitter

In [16]:
def knit_lister(circuit, conloc, tarloc, meas, num_cx):
        """Assembles the list of circuit components in the knitting decomposition

        Parameters:
            circuit (QuantumCircuit): the circuit to be knitted (to match the number of qubits)
            conloc (int): index of the control qubit 
            tarloc (int): index of the target qubit 
            meas (int): which of the num_cx CNOTs is being knitted (this is the classical bit to which the measurement outputs)
            num_cx (int): the number of CNOTs in the original circuit (num_cx+the number of qubits is the number of classical bits necessary)
        Returns:
            circuit list (list): a list of length 6 containing the relevant terms in the knitting decomposition
        """
        qc = QuantumCircuit(circuit.num_qubits, num_cx+circuit.num_qubits)
        circuit_list =[]
        qc.rz(np.pi/2, tarloc)
        qc.rx(np.pi/2, conloc)
        circuit_list.append(qc[:2])
        del(qc)
        qc = QuantumCircuit(circuit.num_qubits, num_cx+circuit.num_qubits)
        qc.rz(-np.pi/2, tarloc)
        qc.rx(-np.pi/2, conloc)
        circuit_list.append(qc[:2])
        del(qc)
        qc = QuantumCircuit(circuit.num_qubits, num_cx+circuit.num_qubits)
        qc.measure(tarloc, meas)
        qc.rx(np.pi, conloc)
        circuit_list.append(qc[:2])
        del(qc)
        qc = QuantumCircuit(circuit.num_qubits, num_cx+circuit.num_qubits)
        qc.measure(tarloc, meas)
        circuit_list.append(qc[:])
        del(qc)
        qc = QuantumCircuit(circuit.num_qubits, num_cx+circuit.num_qubits)
        qc.rz(np.pi, tarloc)
        qc.h(conloc)
        qc.measure(conloc, meas)
        qc.h(conloc)
        circuit_list.append(qc[:4])
        del(qc)
        qc = QuantumCircuit(circuit.num_qubits, num_cx+circuit.num_qubits)
        qc.h(conloc)
        qc.measure(conloc, meas)
        qc.h(conloc)
        circuit_list.append(qc[:3])
        del(qc)
        return circuit_list

In [17]:
def flattener(in_list):
        """flattens a list"""
        out_list = []
        for i in in_list:
            if isinstance(i, list):
                for item in i:
                    out_list.append(item)
            else:
                out_list.append(i)
        return out_list

In [18]:
def my_measure(my_list, conq, tarq, num_qbits, num_cx, num_shots, noise=False):
    """Assembles a quantum circuit from a list of circuit components, runs and measures the circuit and returns the outcomes

    Parameters:
        my_list (list): a list of circuit components
        conq (int): index of the control qubit 
        tarq (int): index of the target qubit
        num_qbits (int): the number of qubits in the original circuit 
        num_cx (int): the number of CNOTs in the original circuit (num_cx+num_qubits is the number of classical bits necessary)
        num_shots (int): the number of shots
        noise (bool): whether to use an ideal or noisy (based on real hardware) backend
    Returns:
        result_dict (dict): a dictionary the keys of which are the measurement outcomes and the items are the counts (contains results
                            of circuit internal measurements, i.e., length of keys is (# of cx) + (# of qubits)
    """
    # initialize circuit and assemble it from components in my_list
    my_circ = QuantumCircuit(num_qbits, num_cx+num_qbits)
    for item in my_list:
        my_circ.append(item)
    my_circ.measure([*range(num_qbits)], [*range(-num_qbits, 0)])

    # select noisy or ideal backend
    if noise:
        backend = FakeWashingtonV2()
    else:
        backend = AerSimulator()
    
    # transpiled circuit 
    pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend, seed_transpiler = 1)
    transpiled_circuit = pass_manager.run(my_circ)
    
    # initialize sampler
    options = {"simulator": {"seed_simulator": 1}}
    sampler = SamplerV2(backend, options=options)
    
    # run job and get results
    job = sampler.run([transpiled_circuit], shots=num_shots)
    result_dict = job.result()[0].data.c.get_counts()
    
    return result_dict

In [19]:
def comb_measure(result_in, conq, tarq, num_cx):
    """Combines quiskit results with circuit internal measurements, e.g., {'0000': 4, '0001': 3} -> {'000': 1}

    Parameters:
        result_in: a dictionary of measurement outcomes with circuit internal measurements
        conq (int): index of the control qubit 
        tarq (int): index of the target qubit 
        num_cx (int): the number of CNOTs in the original circuit (circuit internal measurements are on final num_cx classical bits)
    Returns:
        result_dict (list): a dictionary the keys of which are the measurement outcomes and the items are the counts (contains results
                            of circuit internal measurements)
    """
    result_dict = defaultdict(int)
    for item in result_in:
        end_meas = item[:-num_cx]
        int_meas = item[-num_cx:]
        if end_meas[-conq-1]+end_meas[-tarq-1] == '00':
            if int_meas.count('1') % 2 == 0:
                result_dict[end_meas] += result_in[str(item)]
            else:
                result_dict[end_meas] -= result_in[str(item)]
        elif end_meas[-conq-1]+end_meas[-tarq-1] == '10':
            if int_meas.count('1') % 2 == 0:
                result_dict[end_meas] += result_in[str(item)]
            else:
                result_dict[end_meas] -= result_in[str(item)]
        elif end_meas[-conq-1]+end_meas[-tarq-1] == '01':
            if int_meas.count('1') % 2 == 0:
                result_dict[end_meas] += result_in[str(item)]
            else:
                result_dict[end_meas] -= result_in[str(item)]
        else:
            if int_meas.count('1') % 2 == 0:
                result_dict[end_meas] += result_in[str(item)]
            else:
                result_dict[end_meas] -= result_in[str(item)]
    return result_dict

In [20]:
def circuit_knitter(qc, conq, tarq, num_shots, rand_seed=1, noise=False):
    """Perform circuiting knitting, measure and return outcomes
    
        Parameters:
            qc (QuantumCircuit): the circuit to be knitted
            conq (int): index of the control qubit to be knitted
            tarq (int): index of the target qubit to be knitted
            num_shots (int): the number of measurements to perform
            rand_seed (int): random seed to use for runs to ensure reproducibility 
            noise (bool): whether to use an ideal or noisy (based on real hardware) backend
        Returns:
            res_dict (dict): a sorted dictionary corresponding to the measurement outcomes
    """

    #qc = qc.decompose().decompose()
    
    np.random.seed(rand_seed)    

    # list of positions in circuit of CNOT gates acting on control and target qubits
    cx_list = [i for i, gate in enumerate(qc.data) if gate.operation.name == 'cx' and \
    ((qc.data[i].qubits[0]._index, qc.data[i].qubits[1]._index) == (tarq, conq) or \
    (qc.data[i].qubits[0]._index, qc.data[i].qubits[1]._index) == (conq, tarq))]
    num_cx=len(cx_list)
    
    # list of 6 numerical prefactors of knitting terms -> list of 6**n prefactors for n CNOTS 
    prefac_list = [1/2, 1/2, -1/2, 1/2, -1/2, 1/2]
    prefac_list = [float(np.prod(elem)) for elem in [*product(*[prefac_list for i in range(num_cx)])]]

    # creates a list corresponding to the circuit to be knitted with each target CNOT replaced by a list of the knitting terms
    nest_list = []
    current_cx = 0
    for i, gate in enumerate(qc.data):
        if gate.operation.name == 'cx':
            if (gate.qubits[0]._index, gate.qubits[1]._index) == (tarq, conq):
                nest_list.append(knit_lister(qc, conq, tarq, current_cx, num_cx))
                current_cx += 1
            elif (gate.qubits[0]._index, gate.qubits[1]._index) == (conq, tarq):
                nest_list.append(knit_lister(qc, tarq, conq, current_cx, num_cx))
                current_cx += 1            
            else:
                nest_list.append([gate])
        elif gate.operation.name not in ("measure", "barrier"):
            nest_list.append([gate])
        else:
            None

    # a list of length 6**n containing the instructions to assemble the terms of the decomposition
    circuits = [*product(*nest_list)]
    circuits = [flattener(item) for item in circuits]


    cum_tot = defaultdict(int)
    for i, item in enumerate(prefac_list):
        temp_res_internal_meas = my_measure(circuits[i], conq, tarq, qc.num_qubits, num_cx, num_shots, noise)
        temp_res = comb_measure(temp_res_internal_meas, conq, tarq, num_cx)
        for sub_item in temp_res:
            cum_tot[sub_item] += temp_res[sub_item] * item
    res_dict = {key: cum_tot[key] for key in sorted(cum_tot)}
    
    return res_dict